In [58]:
#import dependencies
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import func


In [34]:
# Step 1: Read CSV Files into DataFrames
industry_groups_df = pd.read_csv('Resources/Industry_Groups.csv')
top_ten_df = pd.read_csv('Resources/top_ten_with_industryID.csv')
top_ten_historic_df = pd.read_csv('Resources/top_ten_historic_5y.csv')
top_ten_historic_df.head()

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,ARB,2018-11-05,17.129999,17.540001,16.370001,17.150000,15.435927,1106127
1,ARB,2018-11-12,17.059999,17.370001,16.655001,17.100000,15.390924,1411720
2,ARB,2018-11-19,17.000000,17.700001,16.330000,17.120001,15.408929,917302
3,ARB,2018-11-26,17.020000,17.559999,16.750000,16.870001,15.183915,1240321
4,ARB,2018-12-03,17.379999,17.490000,16.110001,16.309999,14.679882,1164860


In [56]:
# #trying to see if i had any null values in 'Date' column of top_ten_historic_df
# print(top_ten_historic_df['Ticker'].isnull().sum())

In [55]:
# top_ten_df.info()

In [36]:
#convert datatype to date
top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date'])


In [42]:
top_ten_historic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60881 entries, 0 to 60880
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Ticker     60881 non-null  object        
 1   Date       60881 non-null  datetime64[ns]
 2   Open       60881 non-null  float64       
 3   High       60881 non-null  float64       
 4   Low        60881 non-null  float64       
 5   Close      60881 non-null  float64       
 6   Adj Close  60881 non-null  float64       
 7   Volume     60881 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 3.7+ MB


In [30]:
top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date']).apply(lambda x: x.date())

In [31]:
# print(top_ten_historic_df['Date'].unique())

In [39]:
industry_groups_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   industry_name  27 non-null     object
 1   industry_id    27 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 560.0+ bytes


In [44]:
# Define Database Schema
Base = declarative_base()

class IndustryGroups(Base):
    __tablename__ = 'industry_groups'
    
    industry_name = Column(String)
    industry_id = Column(Integer, primary_key=True)
    

class TopTen(Base):
    __tablename__ = 'top_ten'
    
    ticker = Column(String, primary_key=True)
    company_name = Column(String)
    market_cap = Column(Float)
    industry_id = Column(Integer, ForeignKey('industry_groups.industry_id'))

class TopTenHistoric(Base):
    __tablename__ = 'top_ten_historic'
    
    id = Column (Integer, primary_key=True, autoincrement=True)
    Ticker = Column(String, ForeignKey('top_ten.ticker'))
    Date = Column(Date)
    Open = Column(Float)
    High = Column(Float)
    Low = Column(Float)
    Close = Column(Float)
    Adj_Close = Column('Adj Close', Float)
    Volume = Column(Integer)

#Connect to the Database and Create Tables
# Replace 'sqlite:///asx.db' with your actual database connection string
engine = create_engine('sqlite:///Resources/top_ten_asx.db')
Base.metadata.create_all(engine)


In [45]:
#Insert DataFrames into Database
Session = sessionmaker(bind=engine)
session = Session()

# Insert IndustryGroups
session.bulk_insert_mappings(IndustryGroups, industry_groups_df.to_dict(orient="records"))

# Insert TopTen
session.bulk_insert_mappings(TopTen, top_ten_df.to_dict(orient="records"))

# Insert TopTenHistoric
session.bulk_insert_mappings(TopTenHistoric, top_ten_historic_df.to_dict(orient="records"))





In [46]:
# #commit changes
# session.commit()

In [ ]:
# #Query the Database and Create DataFrame
# # Example: Querying the TopTen table and creating a DataFrame
# top_ten_query = session.query(TopTen).all()
# asx = pd.DataFrame([{
#     'ticker': record.ticker,
#     'company_name': record.company_name,
#     'market_cap': record.market_cap,
#     'industry_id': record.industry_id
# } for record in top_ten_query])

# print(asx)


In [50]:
#import dependencies for inspector
from sqlalchemy import inspect

In [51]:
# Instantiate an Inspector with the engine
inspector = inspect(engine)

In [52]:
# Get a list of table names
table_names = inspector.get_table_names()
print(table_names)

['industry_groups', 'top_ten', 'top_ten_historic']


In [63]:
#Get columns of top_ten_historic
columns = inspector.get_columns('top_ten')
for column in columns:
    print(column['name'])

ticker
company_name
market_cap
industry_id


In [59]:
#reflect database into ORM classes 
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

['industry_groups', 'top_ten', 'top_ten_historic']

In [60]:
#save a reference to the top_ten table as 'top_ten'
top_ten = Base.classes.top_ten

sqlalchemy.ext.automap.top_ten

In [61]:
#create a database session object
session = Session(engine)

In [62]:
#list all of the tickers found in the top_ten table 
session.query(top_ten.ticker).all()

[('360',),
 ('4DS',),
 ('A2M',),
 ('AAC',),
 ('ABB',),
 ('ABV',),
 ('AD8',),
 ('AFI',),
 ('AFP',),
 ('AGL',),
 ('AHL',),
 ('AIA',),
 ('AIZ',),
 ('AKP',),
 ('ALD',),
 ('ALL',),
 ('ALQ',),
 ('ALU',),
 ('ALX',),
 ('AMC',),
 ('AMP',),
 ('ANN',),
 ('ANZ',),
 ('APA',),
 ('APE',),
 ('APM',),
 ('ARB',),
 ('ARG',),
 ('ASX',),
 ('AUB',),
 ('AUI',),
 ('AVH',),
 ('AVJ',),
 ('AX1',),
 ('AXE',),
 ('AZJ',),
 ('BAP',),
 ('BEN',),
 ('BFL',),
 ('BGA',),
 ('BGP',),
 ('BHP',),
 ('BIO',),
 ('BKI',),
 ('BLG',),
 ('BOQ',),
 ('BPT',),
 ('BRG',),
 ('BXB',),
 ('BXN',),
 ('CAR',),
 ('CBA',),
 ('CBO',),
 ('CCO',),
 ('CDA',),
 ('CEN',),
 ('CGC',),
 ('CGF',),
 ('CHC',),
 ('CNU',),
 ('COH',),
 ('COL',),
 ('CPU',),
 ('CSL',),
 ('CTD',),
 ('CUV',),
 ('CVW',),
 ('CWP',),
 ('CWY',),
 ('D2O',),
 ('DBI',),
 ('DDR',),
 ('DDT',),
 ('DHG',),
 ('DMP',),
 ('DOW',),
 ('DTL',),
 ('DTZ',),
 ('DXS',),
 ('DY6',),
 ('EBO',),
 ('EDV',),
 ('ELD',),
 ('ERD',),
 ('EVT',),
 ('EXL',),
 ('FBU',),
 ('FLT',),
 ('FMG',),
 ('FPH',),
 ('FRI',),